#### Step 1 导入相关包

In [1]:
import pandas as pd
import requests
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import openpyxl
import string
import pickle
import json
import os
import time

#### Step 2 加载数据

处理第四个文件

In [6]:
# 打开文件并读取数据
file_index = 4
with open(f'../output/dblp_parsed_{file_index}.json', 'r',encoding='utf-8', errors='ignore') as file:
    pre_data = json.load(file)

In [7]:
len(pre_data)

1000000

In [3]:
if not os.path.exists(f'../openalex_connect_result/dblp_parsed_{file_index}'):
    os.makedirs(f'../openalex_connect_result/dblp_parsed_{file_index}')

#### Step 3 过滤数据

In [4]:
def filter_data(data):
    filtered_data = []
    for item in data:
        if 'ee' in item:
            if isinstance(item['ee'], list):
                for ee in item['ee']:
                    if 'doi' in ee:
                        filtered_data.append(item)
                        break
            elif 'doi' in item['ee']:
                filtered_data.append(item)
        else:
            pass
    return filtered_data

In [5]:
filtered_data = filter_data(pre_data)
print(f'原数据长度： {len(pre_data)}')
print('数据保留率：',len(filtered_data)/len(pre_data))

原数据长度： 1000000
数据保留率： 0.768572


#### Step 4 解析数据

In [6]:
def get_paper_message(url):
    base_url = f"https://api.openalex.org/works/{url}?mailto=1146904171@qq.com"
    # 假设代理服务器地址是 127.0.0.1，端口是 7890
    proxies = {
                "http": "socks5://127.0.0.1:7890",
                "https": "socks5://127.0.0.1:7890",
                }
    try:
        response = requests.get(base_url, timeout=5, proxies=proxies)
        # 检查响应状态码
        if response.status_code == 200:
            try:
                response_json = response.json()
                return response_json
            except Exception as e:
                return None
        elif response.status_code == 404:
            return -1
        else:
            print(f"请求失败，状态码: {response.status_code}")
            return None
    except Exception as e:
        print(f"连接错误: {e}")
        return None

In [7]:
sub_file_index = 1
# 检查已有的文件，找出最大的 sub_file_index
result_dir = f'../openalex_connect_result/dblp_parsed_{file_index}'
existing_files = os.listdir(result_dir)
existing_indexes = []
for file in existing_files:
    if file.startswith('output_') and file.endswith('.json'):
        try:
            index = int(file.split('_')[1].split('.')[0])
            existing_indexes.append(index)
        except ValueError:
            continue
if existing_indexes:
    sub_file_index = max(existing_indexes) + 1

print(f'当前已经处理的文件有： {existing_files}')
print(f'当前的sub_file_index: {sub_file_index}')

当前已经处理的文件有： []
当前的sub_file_index: 1


In [10]:
import random

# 生成 16 个随机 QQ 号列表
mailto_list = []
for _ in range(16):
    qq_number = random.randint(100000, 2999999999)
    mailto_list.append(f"{qq_number}@qq.com")


mailto_index = 0

print(mailto_list)

['519021793@qq.com', '1727255455@qq.com', '1542112286@qq.com', '253223137@qq.com', '2375786039@qq.com', '1875544845@qq.com', '1145630659@qq.com', '2138464596@qq.com', '1024543545@qq.com', '765614139@qq.com', '90828880@qq.com', '2300679545@qq.com', '1652896108@qq.com', '2772069501@qq.com', '613666958@qq.com', '2665194398@qq.com']


#### 批量抓

In [11]:
def batch_get_paper_messages(dois,mailto):
    
    pipe_separated_dois = "|".join(dois)
    base_url = f"https://api.openalex.org/works?filter=doi:{pipe_separated_dois}&per-page=50&mailto={mailto}"
    proxies = {
                "http": "socks5://127.0.0.1:7890",
                "https": "socks5://127.0.0.1:7890",
                }
    try:
        response = requests.get(base_url, timeout=5, proxies=proxies)
        # 检查响应状态码
        if response.status_code == 200:
            try:
                response_json = response.json().get("results", [])
                return response_json
            except Exception as e:
                return None
        else:
            print(f"请求失败，状态码: {response.status_code}")
            return []
    except Exception as e:
        print(f"连接错误: {e}")
        return []

In [12]:
filtered_data[0]

{'type': 'inproceedings',
 'mdate': '2022-04-09',
 'key': 'conf/ecai/0001S20',
 'authors': ['Shakil M. Khan 0001', 'Mikhail Soutchanski'],
 'titles': ['Necessary and Sufficient Conditions for Actual Root Causes.'],
 'pages': ['800-808'],
 'year': ['2020'],
 'booktitle': ['ECAI'],
 'ee': ['https://doi.org/10.3233/FAIA200169'],
 'crossref': ['conf/ecai/2020'],
 'url': ['db/conf/ecai/ecai2020.html#0001S20']}

#### Step 5 开抓数据

In [14]:
mailto_list[mailto_index]

'519021793@qq.com'

In [17]:
start_index = 0
count = 0
linked_data = []

# 从 start_index 开始处理数据
batch_size = 50  # 每批次处理的数量
doi_batch = []
for index, item in enumerate(tqdm(filtered_data)):
    if index < start_index:
        continue
    # 检查 item 中是否包含 'ee' 字段且 'ee' 列表不为空
    if 'ee' in item and len(item['ee']) > 0:
        doi = str(item['ee'][0])
        doi_batch.append(doi)
        if len(doi_batch) >= batch_size:
            mailto = mailto_list[mailto_index]
            paper_infos = batch_get_paper_messages(doi_batch,mailto)
            if paper_infos == []:
                print(f"解析出错")
            else:
                linked_data.extend(paper_infos)
                if len(paper_infos) < 5:
                    print('这次返回地有点少,可能出错了',len(paper_infos))
            doi_batch = []  # 清空 doi_batch 列表
            time.sleep(0.1)
    # 每处理一个元素，计数器加 1
    count += 1
    if count >= 50000:  # 当计数器达到 50000 时, 就保存，设置太大不行，这个colab还是不够稳
        file_name = f'../openalex_connect_result/dblp_parsed_{file_index}/output_{sub_file_index}.json'  # 生成新的文件名
        with open(file_name, 'w') as f:  # 将 linked_data 列表的内容输出到本地文件
            json.dump(linked_data, f)
            print(f'output_{sub_file_index}已成功保存了')
        linked_data = []  # 清空 linked_data 列表
        count = 0  # 计数器重置为 0
        sub_file_index += 1  # 文件索引加 1
        
        # 换个邮箱
        mailto_index = (mailto_index + 1) % len(mailto_list)

  1%|          | 4350/768572 [02:15<7:25:21, 28.60it/s]

解析出错


  1%|          | 4400/768572 [02:16<6:50:52, 31.00it/s]

解析出错


  1%|          | 4450/768572 [02:18<6:30:29, 32.61it/s]

...

 93%|█████████▎| 716000/768572 [8:39:03<54:29, 16.08it/s]  

连接错误: SOCKSHTTPSConnectionPool(host='api.openalex.org', port=443): Read timed out. (read timeout=5)
解析出错


 98%|█████████▊| 749950/768572 [9:03:33<12:56, 23.97it/s]

output_15已成功保存了


100%|██████████| 768572/768572 [9:17:06<00:00, 22.99it/s]


#### 处理剩余的

In [18]:
sub_file_index

16

In [19]:
# 保存最后一批数据
if linked_data:
    file_name = f'../openalex_connect_result/dblp_parsed_{file_index}/output_{sub_file_index}.json'  # 生成新的文件名
    with open(file_name, 'w') as f:  # 将 linked_data 列表的内容输出到本地文件
        json.dump(linked_data, f)
    print(f'output_{sub_file_index}已成功保存了')

output_16已成功保存了
